In [1]:
import pandas as pd
import numpy as np
import tejapi
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS'] # 解決MAC電腦 plot中文問題
plt.rcParams['axes.unicode_minus'] = False
tejapi.ApiConfig.api_key ="Your Key"
tejapi.ApiConfig.ignoretz = True

In [3]:
data=tejapi.get('TWN/ANPRCSTD' ,chinese_column_name=True )
select=data["上市別"].unique()
select=select[1:3]
condition =(data["上市別"].isin(select)) & ( data["證券種類名稱"]=="普通股" )
data=data[condition]
twid=data["證券碼"].to_list()  #取得上市櫃股票證券碼

In [ ]:
df = pd.DataFrame()
for i in twid: #資料筆數超過100萬筆，透過迴圈方式抓取
    df = pd.concat([df, tejapi.get('TWN/APRCD1',  #從TEJ api撈取所需要的資料
                  chinese_column_name = True,
                  paginate = True,
                  mdate = {'gt':'2013-12-31', 'lt':'2022-07-01'},
                  coid=i,
                  opts={'columns':['coid','mdate', 'close_adj' ,'roi' ,'mv', "pbr_tej"]})])

In [4]:
# df = pd.read_csv('alpha.csv', sep=',', encoding='big5')

In [5]:
# df = df.iloc[:,1:]

In [6]:
# df['年月日'] = df['年月日'].map(lambda x: x[:10])

In [7]:
# df['年月日'] = pd.to_datetime(df['年月日'])

In [8]:
df['帳面市值比'] = 1/df['股價淨值比-TEJ']

In [10]:
ME = df.groupby('年月日')['市值(百萬元)'].apply(lambda x: x.median())
ME.name = '市值_中位數'
df = df.merge(ME, on='年月日')
df['市值matrix'] = np.where(df['市值(百萬元)']>df['市值_中位數'], 'B', 'S')

In [13]:
df1 = (df.groupby(['年月日','市值matrix'])['市值(百萬元)'].sum()).reset_index()
df = df.merge(df1, on=['年月日','市值matrix'])
df['weight'] = df['市值(百萬元)_x']/df['市值(百萬元)_y']
df.groupby(['年月日','市值matrix'])['weight'].sum()

In [16]:
df.groupby(['年月日','市值matrix'])['weight'].sum()

年月日         市值matrix
2014-01-02  B           1.0
            S           1.0
2014-01-03  B           1.0
            S           1.0
2014-01-06  B           1.0
                       ... 
2022-07-27  S           1.0
2022-07-28  B           1.0
            S           1.0
2022-07-29  B           1.0
            S           1.0
Name: weight, Length: 4196, dtype: float64

In [37]:
df['return1'] = df['報酬率％']* df['weight']
SMB = df.groupby(['年月日','市值matrix'])['return1'].sum()
SMB.reset_index(inplace=True)
SMB.set_index('年月日',drop=True, inplace=True)
SMB = SMB[SMB['市值matrix']=='S']['return1'] - SMB[SMB['市值matrix']=='B']['return1']
SMB.name = 'SMB'
SMB

In [42]:
SMB

年月日
2014-01-02    0.727428
2014-01-03    1.214870
2014-01-06    0.623517
2014-01-07    0.642468
2014-01-08    0.241418
                ...   
2022-07-25    0.322174
2022-07-26    0.053480
2022-07-27   -0.290234
2022-07-28   -0.106124
2022-07-29    0.065884
Name: SMB, Length: 2098, dtype: float64

In [24]:
a = df.groupby('年月日')['帳面市值比'].quantile(0.7)
a.name = 'BM_0.7'
b = df.groupby('年月日')['帳面市值比'].quantile(0.3)
b.name = 'BM_0.3'
df = df.merge(a, on='年月日')
df = df.merge(b, on='年月日')
df['BM_matrix'] = np.where(df['帳面市值比']>df['BM_0.7'], 'V', (np.where(df['帳面市值比']<df['BM_0.3'],'G', 'N')))

In [45]:
df2 = (df.groupby(['年月日','BM_matrix'])['市值(百萬元)_x'].sum()).reset_index()
df = df.merge(df2, on=['年月日','BM_matrix'])
df['weight2'] = df['市值(百萬元)_x_x']/df['市值(百萬元)_x_y']
df.groupby(['年月日','BM_matrix'])['weight2'].sum()

In [52]:
df['return2'] = df['報酬率％']* df['weight2']
HML = df.groupby(['年月日','BM_matrix'])['return2'].sum()
HML.reset_index(inplace=True)
HML.set_index('年月日',drop=True, inplace=True)
HML = HML[HML['BM_matrix']=='V']['return2'] - HML[HML['BM_matrix']=='G']['return2']
HML.name = 'HML'
HML

In [534]:
HML

年月日
2014-01-02    0.423522
2014-01-03    0.678727
2014-01-06    0.075185
2014-01-07    0.697764
2014-01-08    0.260657
                ...   
2022-07-25    0.767774
2022-07-26    0.787060
2022-07-27   -1.005587
2022-07-28    0.418247
2022-07-29   -0.549442
Name: HML, Length: 2098, dtype: float64

In [535]:
fama = pd.concat([SMB,HML], axis=1)
fama

,SMB,HML
年月日,,
2014-01-02,0.727428,0.423522
2014-01-03,1.214870,0.678727
2014-01-06,0.623517,0.075185
2014-01-07,0.642468,0.697764
2014-01-08,0.241418,0.260657
...,...,...
2022-07-25,0.322174,0.767774
2022-07-26,0.053480,0.787060
2022-07-27,-0.290234,-1.005587


In [79]:
Y9999 = tejapi.get('TWN/APRCD1',  #從TEJ api撈取所需要的資料
                  chinese_column_name = True,
                  paginate = True,
                  mdate = {'gt':'2013-12-31', 'lt':'2022-07-01'},
                  coid='Y9999',
                  opts={'columns':['coid','mdate', 'roi']})

In [81]:
fama = fama.merge(Y9999[['年月日','報酬率％']], on='年月日')
fama.rename(columns = {'報酬率％':'rm'}, inplace=True)
fama.set_index('年月日',drop=True,inplace=True)
fama

In [157]:
fama = fama.loc[:'2022-06-30']

In [161]:
fama

,SMB,HML,rm
年月日,,,
2014-01-02,0.727428,0.423522,0.0120
2014-01-03,1.214870,0.678727,-0.7663
2014-01-06,0.623517,0.075185,-0.5444
2014-01-07,0.642468,0.697764,0.1446
2014-01-08,0.241418,0.260657,0.5135
...,...,...,...
2022-06-24,0.067493,0.737971,0.8360
2022-06-27,0.029549,-0.991883,1.5989
2022-06-28,-0.021501,0.649327,-0.6952


In [160]:
stock = df[['證券代碼', '年月日','報酬率％']]
stock.set_index('年月日', drop=True, inplace=True)
stock = stock.loc[:'2022-06-30']
stock

,證券代碼,報酬率％
年月日,,
2014-01-02,1101,-1.8378
2014-01-02,1102,-1.2953
2014-01-02,1104,0.1770
2014-01-02,1110,1.7143
2014-01-02,1203,0.0000
...,...,...
2022-06-30,8440,9.9836
2022-06-30,8446,-0.2710
2022-06-30,8928,-6.1002


In [220]:
m = pd.date_range('2013-12-31', '2022-07-31', freq='6M').to_list()
X = sm.add_constant(fama)
stock_list = stock['證券代碼'].unique()


In [302]:
b = pd.DataFrame()
for j in stock_list:
    a=[]
    for i in range(len(m)-1):
        try:
            Y = (stock[stock['證券代碼']== j]).loc[m[i]:m[i+1]]
            result = sm.OLS(Y['報酬率％'], X.loc[m[i]:m[i+1]]).fit()
            a.append(result.params[0])
        except:
            pass
    j = str(j)
    c = pd.DataFrame({'證券代碼':([j]*len(a)), 'alpha':a}, index=m[1:len(a)+1])
    b = pd.concat([b,c])
b.index.name = '年月日'

In [323]:
alpha1 = b.groupby('年月日')['alpha'].apply(lambda x : x.quantile(0.8))
alpha1.name = 'alpha0.8'
alpha2 = b.groupby('年月日')['alpha'].apply(lambda x : x.quantile(0.2))
alpha2.name = 'alpha0.2'
b = b.merge(alpha1, on='年月日')
b = b.merge(alpha2, on='年月日')
long = (b.where(b['alpha'] > b['alpha0.8'])).dropna()
short = (b.where(b['alpha'] < b['alpha0.2'])).dropna()

In [532]:
long

,證券代碼,alpha,alpha0.2,alpha0.8
年月日,,,,
2014-06-30,1210,0.216635,-0.142295,0.141690
2014-06-30,1527,0.168521,-0.142295,0.141690
2014-06-30,1582,0.237503,-0.142295,0.141690
2014-06-30,1583,0.585237,-0.142295,0.141690
2014-06-30,1711,0.192855,-0.142295,0.141690
...,...,...,...,...
2022-06-30,8409,0.490293,-0.087633,0.134834
2022-06-30,8433,0.220098,-0.087633,0.134834
2022-06-30,8928,0.279140,-0.087633,0.134834


In [359]:
stock1 = df[['證券代碼','年月日','收盤價(元)']]
stock1.set_index('年月日',drop=True, inplace=True)
stock1 = stock1.loc[:"2022-06-30"]
stock1['證券代碼'] = stock1['證券代碼'].astype('str')

In [468]:
ret = []
for i in range(1, len(m)-1):
    qq = (stock1.loc[m[i]:m[i+1]])['證券代碼'].isin((long.loc[m[i]])['證券代碼'].tolist())
    a = ((stock1.loc[m[i]:m[i+1]])[qq]).groupby('證券代碼')['收盤價(元)'].tail(1).sum()
    b = ((stock1.loc[m[i]:m[i+1]])[qq]).groupby('證券代碼')['收盤價(元)'].head(1).sum()
    c = len((long.loc[m[i]])['證券代碼'].tolist())
    long_ret = ((a/b)-1)/c
    qq1 = (stock1.loc[m[i]:m[i+1]])['證券代碼'].isin((short.loc[m[i]])['證券代碼'].tolist())
    a1 = ((stock1.loc[m[i]:m[i+1]])[qq1]).groupby('證券代碼')['收盤價(元)'].tail(1).sum()
    b1 = ((stock1.loc[m[i]:m[i+1]])[qq1]).groupby('證券代碼')['收盤價(元)'].head(1).sum()
    c1 = len((short.loc[m[i]])['證券代碼'].tolist())
    short_ret = ((a1/b1)-1)/c1
    ret.append(long_ret - short_ret)

In [524]:
ret = pd.DataFrame({'ret':ret}, index=m[2:])

In [479]:
y9999  = tejapi.get('TWN/APRCD1',  #從TEJ api撈取所需要的資料
                  chinese_column_name = True,
                  paginate = True,
                  mdate = {'gt':'2013-12-31', 'lt':'2022-07-01'},
                  coid='Y9999',
                  opts={'columns':['coid','mdate', 'close_adj']})

y9999.set_index('年月日' ,drop=True, inplace=True)

a = []
for i in range(1 , len(m)-1):
    b = (((y9999.loc[m[i]:m[i+1]]).tail(1)['收盤價(元)'].values / (y9999.loc[m[i]:m[i+1]]).head(1)['收盤價(元)'].values) -1)[0]
    a.append(b)

ret['大盤'] = a
ret[['ret', '大盤']].apply(lambda x :x*100)